## Preparação

Este bloco carrega o que será utilizado nas demais atividades do notebook. De modo geral, duas bases são necessárias: registros de vendas e perfil de clientes. Quanto ao perfil dos clientes, para melhor desenvolvimento das atividades, foi criada uma estrutura em árvore para facilitar a consulta do perfil por meio do identificador do cliente.

Quanto ao caminho utilizado como base para carregamento e salvamento de dados, existem dois diferentes por causa do desenvolvimento e execução do projeto em diferentes sistemas operacionais.

In [7]:
import pandas as pd
from typing import List
from datetime import date

BASE_PATH = "/media/bruno/Arquivos/Desenvolvimento/LaranjaLima"

In [56]:
# Carrega bases para processamento
sales = pd.read_excel(f"{BASE_PATH}/data/ecommerce.xlsx", sheet_name="Vendas").sort_values("Data", ascending=True)
customers = pd.read_csv(f"{BASE_PATH}/data/ml/ecommerce - orange - perfil de clientes.csv")

# Organiza perfil de clientes em árvore
customer_groups = {}
for i in range(0, len(customers)):
    customer_groups[customers["Cliente"].values[i]] = customers["Grupo"].values[i]

## Quanto um cliente está disposto a gastar?

A grande motivação é entender quanto um determinado cliente está disposto a gastar em compras em um momento futuro. Em outras palavras, existe um cliente X com um determinado perfil de compra até o último mês. Em 1 mês ele irá gastar mais ou menos? E depois de 2 meses? E se uma campanha de marketing for efetuada?

Estas são as questões guia para a nossa montagem da base, determinando quais informações precisaremos ter para fazer uma análise regressiva. Assim, precisamos montar uma tabela com as seguintes informações:

- Identificação do cliente (apenas para diferenciação)
- Identificação da compra (apenas para diferenciação)
- Grupo do cliente
- Valor da compra em R$
- Dias desde a última compra do cliente
- Desconto total aplicado em %
- Tempo total gasto para finalizar a compra em minutos
- Algum produto estava dentro de uma campanha de marketing?
- O cliente utilizou o PIX para realizar o pagamento?

In [61]:
class DataPurchaseEvent:
    def __init__(self, dt: str, customer: str, coupon: str) -> None:
        self.__date = dt
        self.__customer = customer
        self.__coupon = coupon
        self.__mkt_campaign = None
        self.__total = None
        self.__off = None
        self.__customer_group = None
        self.__time_to_finish = None
        self.__time_last_purchase = None
        self.__bought_in_mkt = None
        self.__pix_payment = None
        self.__categories = None

    def get_date(self) -> str:
        return self.__date

    def get_customer(self) -> str:
        return self.__customer
    
    def get_coupon(self) -> str:
        return self.__coupon

    def set_total(self, total: float) -> None:
        self.__total = round(total, 2)

    def set_off(self, off: float) -> None:
        self.__off = round(off, 2)

    def set_customer_group(self, group: str) -> None:
        self.__customer_group = group

    def set_time_to_finish(self, time: float) -> None:
        self.__time_to_finish = time

    def set_bought_in_mkt(self, in_mkt: bool) -> None:
        self.__bought_in_mkt = in_mkt

    def set_pix_payment(self, pix: bool) -> None:
        self.__pix_payment = pix

    def set_time_last_purchase(self, time: int) -> None:
        self.__time_last_purchase = time

    def set_mkt_campaign(self, mkt: str) -> None:
        self.__mkt_campaign = mkt

    def set_categories(self, categories: str) -> None:
        self.__categories = categories

    def to_dict(self) -> dict:
        return {
            "Data": self.__date,
            "Cliente": self.__customer,
            "Perfil do Cliente": self.__customer_group,
            "Cupom Fiscal": self.__coupon,
            "Campanha em Curso": self.__mkt_campaign,
            "Total Pago (R$)": self.__total,
            "Desconto Total (%)": self.__off,
            "PIX como Forma de Pagamento": self.__pix_payment,
            "Tempo desde a Última Compra (dias)": self.__time_last_purchase,
            "Tempo para Finalizar Compra (min)": self.__time_to_finish,
            "Categorias Compradas": self.__categories,
            "Comprou Item em Campanha de Marketing": self.__bought_in_mkt
        }

In [69]:
# Agrupa os dados de uma compra, mantendo a ordem de data
events: List[DataPurchaseEvent] = []
coupons = []
for i in range(0, len(sales)):
    coupon = sales["Nota"].values[i]
    if coupon not in coupons:
        dt = str(sales["Data"].values[i]).split("T")[0]
        events.append(DataPurchaseEvent(dt, sales["Cliente"].values[i], coupon))
        coupons.append(coupon)

# Para cada compra, calcula os campos necessários
last_purchase = {}
for event in events:
    sub = sales.loc[sales["Nota"] == event.get_coupon()]
    total_off, total, time, in_mkt, with_pix = 0, 0, 0, False, False
    for i in range(0, len(sub)):
        value = sub["Preço de Venda (R$)"].values[i]
        off = sub["Desconto Aplicado (%)"].values[i]
        total = total + value
        total_off = total_off + value / (1 - off / 100)
        time = sub["Tempo de Finalização de compra (min)"].values[i]
        if sub["Categoria"].values[i] == sub["Campanha em Curso"].values[i]: in_mkt = True
        if sub["Compra no PIX"].values[i]: with_pix = True

    # Calcula o tempo desde a última compra, utilizando o dicionário de rastreio
    days = None
    if event.get_customer() not in last_purchase.keys():
        last_purchase[event.get_customer()] = date.fromisoformat(event.get_date())
    else:
        days = (date.fromisoformat(event.get_date()) - last_purchase[event.get_customer()]).days
        last_purchase[event.get_customer()] = date.fromisoformat(event.get_date())

    event.set_total(total)
    event.set_off((1 - total / total_off) * 100)
    event.set_mkt_campaign(sub["Campanha em Curso"].values[0])
    event.set_customer_group(customer_groups[event.get_customer()])
    event.set_time_to_finish(time)
    event.set_bought_in_mkt(in_mkt)
    event.set_pix_payment(with_pix)
    event.set_time_last_purchase(days)
    event.set_categories(";".join(sub["Categoria"].values))

pd.DataFrame([event.to_dict() for event in events]).to_csv(f"{BASE_PATH}/data/ml/ecommerce - python - gasto dos clientes.csv", index=False)